In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from skinet.utils.athena_tools import *
warnings.filterwarnings("ignore")

In [2]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "NanumGothincCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [3]:
ym = '202010' # 전월 
fm = '202009' # 최초 청구월 기준
ft = 20200901 # 최종 통화일 기준

In [4]:
# 마감 데이터 뽑기

conn = athena_connection()
magam = load_from_athena(conn, '''
SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.kunnr as kunnr_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSDT4021 WHERE CTNUM = A.CTNUM AND RQNUM = A.RQNUM AND AMT_TYPE = 'F3' AND MANDT = '100') AS 전자계산서EMAIL
--     , (SELECT GROS_CODE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS GROSCODE
     , a.svc_str
     , a.pay_type
     , a.depo_type
     , a.line_type
     , a.pck_str
--     , (SELECT BIR_DATE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS 생년월일
     , SUBSTR(c.stcd1, 1 ,6) AS B_YMD
     , SUBSTR(c.stcd1, 7 ,1) AS SEX
     , c.stcd2
     , c.j_1kftbus AS uptae
     , c.j_1kftind AS jongmok
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , a.gad_pstlz
     , a.gad_add02
     , a.post_add02
     , b.result1
     , b.result2
     , b.result3
     , b.result4
     , b.result5
     , b.result6
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.sk1_yn
     , b.sk2_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
     , e.ctrt_id
FROM sap.zsdt4013      AS a 
LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam, result1, result2, result3, result4, result5, result6 FROM sap.zsdt3748 WHERE usest = 'X') AS b
      ON a.ctnum = b.ctnum
LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1, j_1kftbus, j_1kftind FROM sap.kna1 WHERE mandt = '100') AS c
      ON a.kunnr = c.kunnr
LEFT JOIN (select distinct ctnum_hash, ctnum, kunnr_hash, kunnr from adt_work.hash_info) d
       ON a.ctnum = d.ctnum_hash
LEFT JOIN (select distinct ctrt_id, ctnum from telepro.tb_nsok_mapping) e
       ON a.ctnum = e.ctnum
  WHERE a.mandt = '100' 
    AND a.stmon = \'{}\'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2');
'''.format(ym)) 

...tmp_db name = tmp_20201119_14_03_24_cacbc7eba8c347bf70ff47c54cb384022301ef9d
Start creating table
drop table if exists default.tmp_20201119_14_03_24_cacbc7eba8c347bf70ff47c54cb384022301ef9d
<< table not exist

        create  table         default.tmp_20201119_14_03_24_cacbc7eba8c347bf70ff47c54cb384022301ef9d
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , a.kunnr as kunnr_hash
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSDT

In [ ]:
conn = athena_connection()
ins = load_from_athena(conn, '''
SELECT DISTINCT b.ctnum
     , b.rqnum
     , b.aedat
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, aedat, ins_user4
             FROM sap.zsdt4150m
            WHERE ins_type IN ('02', '03', '07')) b
LEFT JOIN (SELECT DISTINCT ctnum, rqnum, ins_stat_nm, ins_amt_type_nm FROM mart.zsdr6051) c
       ON b.ctnum = c.ctnum
      AND b.rqnum = c.rqnum
LEFT JOIN (SELECT DISTINCT ctnum, ctnum_old FROM mart.zsdr6001 WHERE ctnum_old <> '') d
       ON b.ctnum = d.ctnum
''') 

In [ ]:
from skinet.utils.athena_tools import *
conn = athena_connection()
tm = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t1 a
LEFT JOIN adt_work.hash_info b
       ON a.kunnr = b.kunnr
GROUP BY a.kunnr
)

SELECT a.*, b.resultcd
  FROM t2 a
LEFT JOIN (SELECT * FROM t1) b
       ON a.kunnr = b.kunnr
      AND a.final_dialdate = b.dialdate;
''') 

In [ ]:
magam.shape

In [ ]:
magam = pd.merge(magam, tm, how='left', left_on='ctnum_hash', right_on='s25csq')

In [ ]:
drop_col = ['kcalss', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 'intro_asp', 'itype_sdesc', 'svc_str', 'pay_type', 'depo_type',
            'result1', 'result2', 'result3', 'result4', 'result5', 'result6']
magam.drop(drop_col, axis=1, inplace=True)

In [ ]:
magam_copy = magam.copy()

In [ ]:
# magam = magam_copy.copy()

In [ ]:
cond0 = magam['first_bildate'].str.contains(fm)                                              # 최초청구월

verify = magam[cond0]

verify['talkenddate'] = verify['talkenddate'].str.replace('/', '', regex=True)
verify['talkenddate'].fillna('00000000', inplace=True)
verify['talkenddate'] = verify['talkenddate'].astype(int)

cond1 = ~verify['ctype_l'].isin(['U00', 'U10', 'U20', 'U30', 'U40', 'U50', 'R00', 'W00'])           # 채널(대)
cond2 = ~verify['ctype_m'].isin(['L07'])                                                            # 채널 (중)
cond3 = ~verify['ctype_s'].isin(['W05'])                                                            # 채널 (소)
cond4 = verify['conn_type'].isin(['A01', 'A02'])                                                    # CMS, Non CMS
cond5 = ~verify['itype_l'].isin(['2', '3'])                                                         # 업종(대)
cond6 = ~verify['itype_m'].isin(['22'])                                                             # 업종(중)
cond7 = ~verify['itype_s'].isin(['160', '217', '222', '223'])                                       # 업종(소)
cond8 = ~((verify['conn_type'] == 'A02') & (verify['itype_m'].isin(['21'])))
# cond9 = ~((verify['conn_type'] == 'A02') & 
#           (verify['itype_s'].isin(['128', '132', '133', '135', '136', '137', '155', '273', 
#                                   '278', '288', '289', '290', '291', '292'])))
cond10 = verify['ctrt_id'].isnull()                                                                  # N사 고객
cond11 = verify['gros_code'] == ' '
cond12 = verify['gross_type'] != 'Y'
# cond13 = magam['adt1_yn']
cond14 = verify['tm_yn'] == 'Y'

cond15 = ~((verify['itype_l'] == '1') & ((verify['monexp_amt'] < 300) | (verify['monexp_amt'] > 4000))) # 월정료기준
cond16 = ~((verify['itype_l'] == '4') & ((verify['monexp_amt'] < 300) | (verify['monexp_amt'] > 1500))) 

cond21 = (verify['dialresultcd'] != 'SE01')
cond22 = (verify['talkenddate'] < ft)

In [ ]:
# 삭제 제외 전화번호 리스트 
no_delete = [
'00','01','02','03','010','0101','01111','010000','010226','0100000','0101111','0105256','0109152','0109364','01000000','01001638','02000000','010000000','010010010','010111111'
,'011111111','020000000','021111111','0100000000','0100000001','0100100000','0101010101','0101111111','0101111234','0101231234','0101234567','0110000000','0111110000','0111234567','0200000000'
,'0201021021','0202222222','0111111111','0211000000','0211111112','0212341234','0211112222','0222222222','0510000000','01000000000','01000000001','01000000250','01000000602','01000001111','01000009999'
,'01000010001','01000090000','01000112111','01001000000','01001000900','01001010101','01001111111','01010000000','01010001000','01011110000','01011111018','01011111110','01011111111'
,'01001001010','01011111112','01011111118','01011111234','01011111313','01011112222','01012341234','01012345678','01022222222','01100000000','01101111111','01101234567','01111111111'
,'01111112222','01111113333','01112311231','03200000000','05100000000','05100011111','05101111111','05101112222','010000000000','010111111111','011111111111','0100000000000','01000000000000'
,'010000000000000','0','1','2','3','10','101','1111','10000','10226','100000','101111','105256','109152','109364','1000000','1001638','2000000','10000000','10010010','10111111','11111111'
,'20000000','21111111','100000000','100000001','100100000','101010101','101111111','101111234','101231234','101234567','110000000','111110000','111234567','200000000','201021021','202222222'
,'111111111','211000000','211111112','212341234','211112222','222222222','510000000','1000000000','1000000001','1000000250','1000000602','1000001111','1000009999','1000010001','1000090000'
,'1000112111','1001000000','1001000900','1001010101','1001111111','1010000000','1010001000','1011110000','1011111018','1011111110','1011111111','1001001010','1011111112','1011111118','1011111234'
,'1011111313','1011112222','1012341234','1012345678','1022222222','1100000000','1101111111','1101234567','1111111111','1111112222','1111113333','1112311231','3200000000','5100000000','5100011111'
,'5101111111','5101112222','10000000000','10111111111','11111111111','100000000000','1000000000000','10000000000000']

In [ ]:
def valid_number(x):
      nums = list(str(x)) 
      n = len(nums)
      answer=[]
      i = 3
      
      while i <= n-3:
            if int(nums[i]) - int(nums[i+1]) == int(nums[i+1]) - int(nums[i+2]):
                  answer.append(True)
            else:
                  answer.append(False)
            i += 1
      
      if (all(answer) == True) or (str(x) in (no_delete)):
            return False
      else:
            return True     

In [ ]:
verify['필터조건'] = 0
verify.loc[~cond1 | ~cond2 | ~cond3 ,'필터조건'] = 1
verify.loc[(verify['필터조건']==0) & (~cond4), '필터조건'] = 2
verify.loc[(verify['필터조건']==0) & (~cond5 | ~cond6 | ~cond7 | ~cond8) , '필터조건'] = 3
verify.loc[(verify['필터조건']==0) & (~cond10) , '필터조건'] = 4
verify.loc[(verify['필터조건']==0) & (~cond11| ~cond12) , '필터조건'] = 5
verify.loc[(verify['필터조건']==0) & (~cond14) , '필터조건'] = 6
verify.loc[(verify['필터조건']==0) & (~cond15 | ~cond16) , '필터조건'] = 7

verify = verify.sort_values(by=['conn_type', 'ctnum'], axis=0, ascending=[True, False])                                     # 고객코드 중복제거
verify.loc[(verify['필터조건']==0) & (verify[verify['필터조건']==0].duplicated(subset='kunnr', keep='first')) , 
           '필터조건'] = 8

        
verify['telf2'] = verify['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
verify['telf2'] = np.where(verify['telf2'].str[0] != '0', "0"+verify['telf2'], verify['telf2'])
verify['telf2'] = verify['telf2'].str.replace(' ', '')
verify['telf2'] = verify['telf2'].str.replace('(', '')
verify['telf2'] = verify['telf2'].str.replace(')', '')

verify['telf_valid_1'] = verify['telf2'].str.contains('^01([0|1|6|7|8|9]?)-?([0-9]{3,4})-?([0-9]{4})$', regex=True)
verify['telf_valid_2'] = verify['telf2'].apply(valid_number)
verify['telf_valid_final'] = np.where((verify['telf_valid_1'] == True) & (verify['telf_valid_2'] == True), True, False)
verify['telf_valid_final'] = verify['telf_valid_final'].map({True : 1, False : 0})

verify = verify.sort_values(by=['conn_type', 'ctnum'], axis=0, ascending=[True, False])                                   # 휴대폰번호 중복제거

verify.loc[(verify['필터조건']==0) & (verify[(verify['필터조건']==0) & ((verify['telf_valid_final']==1))].duplicated(subset='telf2', keep='first'))
           , '필터조건'] = 9

verify.loc[(verify['필터조건']==0) & (~cond21) , '필터조건'] = 10
verify.loc[(verify['필터조건']==0) & (~cond22) , '필터조건'] = 11

In [ ]:
filter1 = ~verify['gad_name'].str.contains('\(사\)')
filter2 = ~verify['gad_name'].str.contains('사단')
filter3 = ~verify['gad_name'].str.contains('\(재\)')
filter4 = ~verify['gad_name'].str.contains('재단')

filter5 = ~((verify['gad_name'].str.contains('건물')) & ((verify['uptae'].str.contains('건물')) | (verify['jongmok'].str.contains('건물'))))
filter6 = ~((verify['gad_name'].str.contains('건물')) & ((verify['uptae'].str.contains('대지')) | (verify['jongmok'].str.contains('대지'))))
filter7 = ~((verify['gad_name'].str.contains('건물')) & ((verify['uptae'].str.contains('부동산')) | (verify['jongmok'].str.contains('부동산'))))
filter8 = ~((verify['gad_name'].str.contains('건물')) & ((verify['uptae'].str.contains('빌딩')) | (verify['jongmok'].str.contains('빌딩'))))
filter9 = ~((verify['gad_name'].str.contains('건물')) & ((verify['uptae'].str.contains('임대')) | (verify['jongmok'].str.contains('임대'))))
filter10 = ~((verify['gad_name'].str.contains('건물')) & ((verify['uptae'].str.contains('점포')) | (verify['jongmok'].str.contains('점포'))))

filter11 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('건물')) | (verify['jongmok'].str.contains('건물'))))
filter12 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('건설')) | (verify['jongmok'].str.contains('건설'))))
filter13 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('부동산')) | (verify['jongmok'].str.contains('부동산'))))
filter14 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('시설')) | (verify['jongmok'].str.contains('시설'))))
filter15 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('임대')) | (verify['jongmok'].str.contains('임대'))))
filter16 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('주차장')) | (verify['jongmok'].str.contains('주차장'))))
filter17 = ~((verify['gad_name'].str.contains('관리')) & ((verify['uptae'].str.contains('주택')) | (verify['jongmok'].str.contains('주택'))))

filter18 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('건설')) | (verify['jongmok'].str.contains('건설'))))
filter19 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('건축')) | (verify['jongmok'].str.contains('건축'))))
filter20 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('관리')) | (verify['jongmok'].str.contains('관리'))))
filter21 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('부동산')) | (verify['jongmok'].str.contains('부동산'))))
filter22 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('상가')) | (verify['jongmok'].str.contains('상가'))))
filter23 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('임대')) | (verify['jongmok'].str.contains('임대'))))
filter24 = ~((verify['gad_name'].str.contains('빌딩')) & ((verify['uptae'].str.contains('주택')) | (verify['jongmok'].str.contains('주택'))))

filter25 = ~((verify['gad_name'].str.contains('빌라')) & ((verify['uptae'].str.contains('건설')) | (verify['jongmok'].str.contains('건설'))))
filter26 = ~((verify['gad_name'].str.contains('빌라')) & ((verify['uptae'].str.contains('관리')) | (verify['jongmok'].str.contains('관리'))))
filter27 = ~((verify['gad_name'].str.contains('빌라')) & ((verify['uptae'].str.contains('부동산')) | (verify['jongmok'].str.contains('부동산'))))
filter28 = ~((verify['gad_name'].str.contains('빌라')) & ((verify['uptae'].str.contains('아파트')) | (verify['jongmok'].str.contains('아파트'))))
filter29 = ~((verify['gad_name'].str.contains('빌라')) & ((verify['uptae'].str.contains('임대')) | (verify['jongmok'].str.contains('임대'))))
filter30 = ~((verify['gad_name'].str.contains('빌라')) & ((verify['uptae'].str.contains('주택')) | (verify['jongmok'].str.contains('주택'))))

filter31 = ~((verify['gad_name'].str.contains('상가')) & ((verify['uptae'].str.contains('관리')) | (verify['jongmok'].str.contains('관리'))))
filter32 = ~((verify['gad_name'].str.contains('상가')) & ((verify['uptae'].str.contains('번영')) | (verify['jongmok'].str.contains('번영'))))
filter33 = ~((verify['gad_name'].str.contains('상가')) & ((verify['uptae'].str.contains('부동산')) | (verify['jongmok'].str.contains('부동산'))))
filter34 = ~((verify['gad_name'].str.contains('상가')) & ((verify['uptae'].str.contains('상가')) | (verify['jongmok'].str.contains('상가'))))
filter35 = ~((verify['gad_name'].str.contains('상가')) & ((verify['uptae'].str.contains('임대')) | (verify['jongmok'].str.contains('임대'))))
filter36 = ~((verify['gad_name'].str.contains('상가')) & ((verify['uptae'].str.contains('주차')) | (verify['jongmok'].str.contains('주차'))))

filter37 = ~((verify['gad_name'].str.contains('아파트')) & ((verify['uptae'].str.contains('건설')) | (verify['jongmok'].str.contains('건설'))))
filter38 = ~((verify['gad_name'].str.contains('아파트')) & ((verify['uptae'].str.contains('건축')) | (verify['jongmok'].str.contains('건축'))))

filter39 = ~((verify['gad_name'].str.contains('양식장')) & (verify['itype_m'] != '02'))

filter40 = ~verify['gad_name'].str.contains('원룸')

filter41 = ~((verify['gad_name'].str.contains('조합')) & ((verify['uptae'].str.contains('건설')) | (verify['jongmok'].str.contains('건설'))))
filter42 = ~((verify['gad_name'].str.contains('조합')) & ((verify['uptae'].str.contains('재건축')) | (verify['jongmok'].str.contains('재건축'))))
filter43 = ~((verify['gad_name'].str.contains('조합')) & ((verify['uptae'].str.contains('주택')) | (verify['jongmok'].str.contains('주택'))))

filter44 = ~verify['gad_name'].str.contains('주차장')
filter45 = ~verify['gad_name'].str.contains('타이어 뱅크')
filter46 = ~verify['gad_name'].str.contains('타이어뱅크')
filter47 = ~verify['gad_name'].str.contains('노인정')
filter48 = ~verify['gad_name'].str.contains('기숙사')

filter49 = ~((verify['gad_name'].str.contains('농장')) & (verify['itype_m'] != '02') & (~(verify['uptae'].str.contains('음식')) & (~verify['jongmok'].str.contains('음식'))))

filter50 = ~verify['gad_name'].str.contains('보험')
filter51 = ~verify['gad_name'].str.contains('복지')
# filter52 = ~verify['gad_name'].str.contains('게스트하우스')
# filter53 = ~verify['gad_name'].str.contains('펜션')
# filter54 = ~verify['gad_name'].str.contains('고시원')
# filter55 = ~verify['gad_name'].str.contains('고시텔')

filter52 = ~((verify['uptae'].str.contains('복지')) | (verify['jongmok'].str.contains('복지')))
# filter57 = ~((verify['uptae'].str.contains('민박')) | (verify['jongmok'].str.contains('민박')))
# filter58 = ~((verify['uptae'].str.contains('여관')) | (verify['jongmok'].str.contains('여관')))

# filter59 = ~((verify['itype_m'] != '02') & ((verify['uptae'].str.contains('숙박')) | (verify['jongmok'].str.contains('숙박'))) & (~(verify['uptae'].str.contains('음식')) & (~verify['jongmok'].str.contains('음식'))))
# filter60 = ~((verify['itype_m'] != '02') & ((verify['uptae'].str.contains('히우스')) | (verify['jongmok'].str.contains('하우스')))) 
# filter61 = ~((verify['itype_m'] != '02') & ((verify['uptae'].str.contains('호텔')) | (verify['jongmok'].str.contains('호텔'))) & (~(verify['uptae'].str.contains('비품')) & (~verify['jongmok'].str.contains('비품'))))
# filter62 = ~((verify['itype_m'] != '02') & ((verify['uptae'].str.contains('호텔')) | (verify['jongmok'].str.contains('호텔'))) & (~(verify['uptae'].str.contains('애견')) & (~verify['jongmok'].str.contains('애견'))))
# filter63 = ~((verify['itype_m'] != '02') & ((verify['uptae'].str.contains('호텔')) | (verify['jongmok'].str.contains('호텔'))) & (~(verify['uptae'].str.contains('용품')) & (~verify['jongmok'].str.contains('용품'))))


filter53 = ~((verify['itype_l'] != '4') & ((verify['uptae'].str.contains('다가구')) | (verify['jongmok'].str.contains('다가구'))))
filter54 = ~((verify['itype_l'] != '4') & ((verify['uptae'].str.contains('빌딩')) | (verify['jongmok'].str.contains('빌딩'))))
filter55 = ~((verify['itype_l'] != '4') & ((verify['uptae'].str.contains('빌라')) | (verify['jongmok'].str.contains('빌라'))))
filter56 = ~((verify['itype_l'] != '4') & ((verify['uptae'].str.contains('아파트')) | (verify['jongmok'].str.contains('아파트'))))

filter57 = ~((verify['uptae'].str.contains('가축')) | (verify['jongmok'].str.contains('가축')))
# filter69 = ~((verify['uptae'].str.contains('게스트하우스')) | (verify['jongmok'].str.contains('게스트하우스')))
# filter70 = ~((verify['uptae'].str.contains('고시원')) | (verify['jongmok'].str.contains('고시원')))
# filter71 = ~((verify['uptae'].str.contains('고시텔')) | (verify['jongmok'].str.contains('고시텔')))
filter58 = ~((verify['uptae'].str.contains('과수원')) | (verify['jongmok'].str.contains('과수원')))
filter59 = ~((verify['uptae'].str.contains('관공서')) | (verify['jongmok'].str.contains('관공서')))
filter60 = ~((verify['uptae'].str.contains('국가기관')) | (verify['jongmok'].str.contains('국가기관')))
filter61 = ~((verify['uptae'].str.contains('글램핑')) | (verify['jongmok'].str.contains('글램핑')))

filter62 = ~((verify['uptae'] == '글램핑') | (verify['jongmok'] == '글램핑'))

filter63 = ~((verify['uptae'].str.contains('농원')) | (verify['jongmok'].str.contains('농원')))
filter64 = ~((verify['uptae'].str.contains('농장')) | (verify['jongmok'].str.contains('농장')))
filter65 = ~((verify['uptae'].str.contains('돈사')) | (verify['jongmok'].str.contains('돈사')))
filter66 = ~((verify['uptae'].str.contains('묘목')) | (verify['jongmok'].str.contains('묘목')))
filter67 = ~((verify['uptae'].str.contains('보험')) | (verify['jongmok'].str.contains('보험')))
filter68 = ~((verify['uptae'].str.contains('비영리')) | (verify['jongmok'].str.contains('비영리')))
filter69 = ~((verify['uptae'].str.contains('생화')) | (verify['jongmok'].str.contains('생화')))
filter70 = ~((verify['uptae'].str.contains('손해사정')) | (verify['jongmok'].str.contains('손해사정')))
filter71 = ~((verify['uptae'].str.contains('식물')) | (verify['jongmok'].str.contains('식물')))
filter72 = ~((verify['uptae'].str.contains('양계')) | (verify['jongmok'].str.contains('양계')))
filter73 = ~((verify['uptae'].str.contains('양봉')) | (verify['jongmok'].str.contains('양봉')))

filter74 = ~((verify['itype_m'] != '02') & ((verify['uptae'].str.contains('양식')) | (verify['jongmok'].str.contains('양식'))) & (~(verify['uptae'].str.contains('음식')) & (~verify['jongmok'].str.contains('음식'))))

filter75 = ~((verify['uptae'].str.contains('양어장')) | (verify['jongmok'].str.contains('양어장')))
filter76 = ~((verify['uptae'].str.contains('어업')) | (verify['jongmok'].str.contains('어업')))
filter77 = ~((verify['uptae'].str.contains('원룸')) | (verify['jongmok'].str.contains('원룸')))
filter78 = ~((verify['uptae'].str.contains('조경')) | (verify['jongmok'].str.contains('조경')))
filter79 = ~((verify['uptae'].str.contains('축사')) | (verify['jongmok'].str.contains('축사')))

filter80 = ~(((verify['uptae'].str.contains('캠핑')) | (verify['jongmok'].str.contains('캠핑'))) & (~(verify['uptae'].str.contains('용품')) & (~verify['jongmok'].str.contains('용품'))))
filter81 = ~(((verify['uptae'].str.contains('캠핑')) | (verify['jongmok'].str.contains('캠핑'))) & (~(verify['uptae'].str.contains('장비')) & (~verify['jongmok'].str.contains('장비'))))

# filter96 = ~((verify['uptae'].str.contains('펜션')) | (verify['jongmok'].str.contains('펜션')))
filter82 = ~((verify['uptae'].str.contains('화훼')) | (verify['jongmok'].str.contains('화훼')))

filter83 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('화원') & (verify['itype_m'] != '02')))
filter84 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('화원') & (verify['itype_m'] != '06')))

# filter100 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('단란주점')))
# filter101 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('유흥주점')))
# filter102 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('단란주점')) | (verify['jongmok'].str.contains('단란주점'))))
# filter103 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('유흥주점')) | (verify['jongmok'].str.contains('유흥주점'))))
# filter104 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('노래')))
# filter105 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('노래')) | (verify['jongmok'].str.contains('노래'))))

# filter106 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'] == '도매') | (verify['jongmok'] == '도매')))
# filter107 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'] == '도매업') | (verify['jongmok'] == '도매업')))

# filter108 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('창고')) | (verify['jongmok'].str.contains('창고'))))
# filter109 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('중개인')))
# filter110 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('가구')) | (verify['jongmok'].str.contains('가구'))))

# filter111 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('가구')))
# filter112 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('테크')))
# filter113 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('정밀')))
# filter114 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('건재')))
# filter115 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('물류')))
# filter116 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('철강')))
# filter117 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('자원')))
# filter118 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('고물')))
# filter119 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('기계')))
# filter120 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('정공')))
# filter121 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('석재')))
# filter122 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('연마')))
# filter123 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('금형')))
# filter124 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('공장')))
# filter125 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('창고')))
# filter126 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('제조')))
# filter127 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('요양원')))
# filter128 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('리싸이클')))
# filter129 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('사우나')))
# filter130 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('찜질')))
# filter131 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('목욕')))
# filter132 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('폐차')))
# filter133 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('스텐')))
# filter134 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('산업')))
# filter135 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('금속')))
# filter136 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('무역')))
# filter137 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('철재')))
# filter138 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('가설')))
# filter139 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('공업사')))
# filter140 = ~((verify['conn_type'] == 'A02') & (verify['gad_name'].str.contains('택배')))

# filter141 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('무역')) | (verify['jongmok'].str.contains('무역'))))
# filter142 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('공장')) | (verify['jongmok'].str.contains('공장'))))
# filter143 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('제조')) | (verify['jongmok'].str.contains('제조'))))
# filter144 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('철근')) | (verify['jongmok'].str.contains('철근'))))
# filter145 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('사우나')) | (verify['jongmok'].str.contains('사우나'))))
# filter146 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('찜질')) | (verify['jongmok'].str.contains('찜질'))))
# filter147 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('목욕')) | (verify['jongmok'].str.contains('목욕'))))
# filter148 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('목재')) | (verify['jongmok'].str.contains('목재'))))
# filter149 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('연구')) | (verify['jongmok'].str.contains('연구'))))
# filter150 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('폐차')) | (verify['jongmok'].str.contains('폐차'))))
# filter151 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('고물')) | (verify['jongmok'].str.contains('고물'))))
# filter152 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('재활용')) | (verify['jongmok'].str.contains('재활용'))))
# filter153 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('고철')) | (verify['jongmok'].str.contains('고철'))))
# filter154 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('기계')) | (verify['jongmok'].str.contains('기계'))))
# filter155 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('농기구')) | (verify['jongmok'].str.contains('농기구'))))
# filter156 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('모델하우스')) | (verify['jongmok'].str.contains('모델하우스'))))
# filter157 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('사격')) | (verify['jongmok'].str.contains('사격'))))
# filter158 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('파지')) | (verify['jongmok'].str.contains('파지'))))
# filter159 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('비철')) | (verify['jongmok'].str.contains('비철'))))
# filter160 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('제생영')) | (verify['jongmok'].str.contains('제생영'))))
# filter161 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('산업')) | (verify['jongmok'].str.contains('산업'))))
# filter162 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('가공')) | (verify['jongmok'].str.contains('가공'))))

# filter163 = ~((verify['conn_type'] == 'A02') & (verify['itype_m'] != '05') & ((verify['uptae'].str.contains('장비')) | (verify['jongmok'].str.contains('장비'))) & ((verify['uptae'] == '소매') | (verify['jongmok'] == '소매')))

# filter164 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('태양광')) | (verify['jongmok'].str.contains('태양광'))))
# filter165 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('절')) | (verify['jongmok'].str.contains('절'))))
# filter166 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('법당')) | (verify['jongmok'].str.contains('법당'))))
# filter167 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('불교')) | (verify['jongmok'].str.contains('불교'))))
# filter168 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('사찰')) | (verify['jongmok'].str.contains('사찰'))))
# filter169 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('용접')) | (verify['jongmok'].str.contains('용접'))))
# filter170 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('대중탕')) | (verify['jongmok'].str.contains('대중탕'))))
# filter171 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('인쇄')) | (verify['jongmok'].str.contains('인쇄'))))
# filter172 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('폐기물')) | (verify['jongmok'].str.contains('폐기물'))))
# filter173 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('토목')) | (verify['jongmok'].str.contains('토목'))))

# filter174 = ~((verify['conn_type'] == 'A02') & (verify['itype_m'] != '05') & ((verify['uptae'].str.contains('전자상거래')) | (verify['jongmok'].str.contains('전자상거래'))) & ((verify['uptae'] == '소매') | (verify['jongmok'] == '소매')) & (verify['gad_add01'].str.contains('[^1]층', regex=True)))

# filter175 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('종자')) | (verify['jongmok'].str.contains('종자'))))
# filter176 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('반도체')) | (verify['jongmok'].str.contains('반도체'))))
# filter177 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('석재')) | (verify['jongmok'].str.contains('석재'))))
# filter178 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('건설')) | (verify['jongmok'].str.contains('건설'))))
# filter179 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('광업')) | (verify['jongmok'].str.contains('광업'))))
# filter180 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('분양')) | (verify['jongmok'].str.contains('분양'))))
# filter181 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('자재')) | (verify['jongmok'].str.contains('자재'))))
# filter182 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('합판')) | (verify['jongmok'].str.contains('합판'))))
# filter183 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('요양원')) | (verify['jongmok'].str.contains('요양원'))))
# filter184 = ~((verify['conn_type'] == 'A02') & ((verify['uptae'].str.contains('구내식당')) | (verify['jongmok'].str.contains('구내식당'))))

# filter185 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('지하')))
# filter186 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('지층')))
# filter187 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('[Bb]\d', regex=True)))
# filter188 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('도매시장')))
# filter189 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('어시장')))
# filter190 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('산물시장')))
# filter191 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('종합시장')))
# filter192 = ~((verify['conn_type'] == 'A02') & (verify['gad_add01'].str.contains('수산시장')))

# filter193 = ~((verify['conn_type'] == 'A02') & (verify['itype_s'] == '062') & (verify['gad_add01'].str.contains('[^1]층', regex=True)))


In [ ]:
temp1 = verify.copy()

In [ ]:
verify.shape

In [ ]:
# 필터 돌리기

for i in range(1, 85):
      verify = verify[globals()['filter' + str(i)]]

In [ ]:
verify.shape

In [ ]:
verify = pd.DataFrame(verify['ctnum'].astype(str))
verify['name_fine'] = 1
final = temp1.merge(verify, how='left', on='ctnum')
final.shape

In [ ]:
final.loc[(final['필터조건']==0) & (final['name_fine'].isnull()==True), '필터조건'] = 13

In [ ]:
final['필터조건'].value_counts(dropna=False)

In [ ]:
final.columns

In [ ]:
final.drop(['kunnr_hash', 'pck_str', 'b_ymd', 'sex', 'stcd2', 's25csq',
           'telf_valid_1', 'telf_valid_2', 'telf_valid_final', 'name_fine'], axis=1, inplace=True)

In [ ]:
final = final.rename(columns={
      'kunnr' : '고객번호',
      'ctnum' : '계약번호',
      'cstat' : '계약상태',
      'gad_name' : '상호',
      'gad_start' : '경비개시일',
      'conn_start' : '계약개시일',
      'conn_end' : '계약종료일',
      'monexp_amt' : '월정료',
      'conn_type' : '계약타입',
      'ctype_l' : '채널(대)',
      'ctype_m' : '채널(중)',
      'ctype_s' : '채널(소)',
      'itype_l' : '업종(대)',
      'itype_m' : '업종(중)',
      'itype_s' : '업종(소)',
      'name1' : '이름1',
      'gad_add01' : '경비물건주소',
      'post_add01' : '우편물수령지주소1',
      'first_bildate' : '최초청구일',
      'telf2' : '전화번호',
      'line_type' : '관제회선구분',
      'uptae' : '업태',
      'jongmok' : '종목',
      'gros_code' : '그로스코드',
      'gross_type' : '그로스타입',
      'kvgr4' : '세그먼트',
      'gad_pstlz' : '우편번호',
      'gad_add02' : '경비물건주소상세',
      'post_add02' : '우편물수령지주소2',
      'sms_yn' : 'SMS동의',
      'email_yn' : 'EMAIL동의',
      'tm_yn' : 'TM동의',
      'dm_yn' : 'DM동의',
      'sk1_yn' : 'SK1동의',
      'sk2_yn' : 'SK2동의',
      'adt1_yn' : '마케팅수신동의',
      'aedat' : '작업일자',
      'aenam' : '변경자',
      'ctrt_id' : 'n사계약번호',
      'rqnum' : '변경요청번호',
      'ins_type' : '보험구분',
      'ins_amt' : '보험금액',
      'ins_amt_type' : '배상액구분',
      'ins_payment' : '보험납부금액',
      'ins_stat' : '보험현상태구분',
      'ins_stat_nm' : '보험현상태구분이름',
      'ins_amt_type_nm' : '배상액구분이름',
      'talkenddate' : '최종접촉일',
      'contactendtime' : '최종접촉시간',
      'dialresultcd' : '최종접촉결과',
      'contact_cnt' : '접촉회차'
})

In [ ]:
final = final[
    ['필터조건', '고객번호', '계약번호', '계약상태', '상호', '경비개시일', '계약개시일', '계약종료일', 
     '월정료', '계약타입', '채널(대)', '채널(중)', '채널(소)', '업종(대)', '업종(중)', '업종(소)', 
           '이름1', '경비물건주소', '우편물수령지주소1', '최초청구일', '전화번호',
           '관제회선구분', '업태', '종목', '그로스코드', '그로스타입', '세그먼트', '우편번호', '경비물건주소상세',
           '우편물수령지주소2', 'SMS동의', 'EMAIL동의', 'TM동의', 'DM동의', 'SK1동의', 'SK2동의', '마케팅수신동의',
           '작업일자', '변경자', 'n사계약번호', '최종접촉일', '최종접촉시간', '최종접촉결과',
           '접촉회차', 'ctnum_hash']
]

In [ ]:
final.to_csv('검증.csv', encoding='utf-8-sig')